In [ ]:
import os
import glob
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils import set_matplotlib_configuration

SAVE_FOLDER = os.getcwd() + "/images"

## CoLEXT

This notebook is for plotting results that were obtained on CoLEXT

In [ ]:
PLOTTING_KWARGS, SAVEFIG_KWARGS = set_matplotlib_configuration(8.1)
os.chdir("../outputs/colext_measure_time")

In [ ]:
all_data = []
for folder in os.listdir():
    device = folder.split("-")[1]
    for file in glob.glob(f"{folder}/*.json"):
        with open(file, "r") as fp:
            data = json.load(fp)

        if "profiler" not in data["config"]:
            print(file)
            continue
        all_data.append({
            "times": np.mean(data["times"]),
            "size": data["config"]["dataset"]["dataset_size"],
            "device": device,
            "cuda": data["GPU"],
            "profiler": data["config"]["profiler"],
            "model": data["config"]["model"]["_target_"]
        })

In [ ]:
df = pd.DataFrame(all_data)
df = df[~((df["profiler"] == "coleds") & (df["model"] == "src.models.clmean_model.ClMeanModel"))]

In [ ]:
def map_profiler_name(row):
    if row["profiler"] == "coleds":
        return "CoLEDS"
    elif row["profiler"] == "wdp":
        return "WDP"
    elif row["profiler"] == "lgp":
        return "LgP"
    elif row["profiler"] == "es":
        return "AESP" if row["model"] == "src.models.models.VariationalAutoencoder" else "REPA"

In [ ]:
df["profiler"] = df.apply(map_profiler_name, axis=1)

In [ ]:
del PLOTTING_KWARGS["error_kw"]

In [ ]:
DEVICE_TO_NAME_MAPPING = {
    "jao1": "Jetson\nAGX Orin$^*$",
    "jon1": "Jetson\nOrin Nano$^*$",
    "jxn1": "Jetson\nXavier NX$^*$",
    "lp1": "Latte Panda\nDelta 3",
    "op1": "OrangePi\n5B",
}

In [ ]:
devices = list(df["device"].unique())
devices.remove("jn3")

fig, ax = plt.subplots(1, len(devices), figsize=(5.1, 1), sharex=True, sharey=True)
for idx, device in enumerate(sorted(devices)):
    ax[idx].set_title(DEVICE_TO_NAME_MAPPING[device])
    device_df = df[df["device"] == device]
    device_df.pivot(index="size", columns="profiler", values="times")[["WDP", "LgP", "CoLEDS", "AESP", "REPA"]].plot(ax=ax[idx], legend=None, **PLOTTING_KWARGS, linewidth=0.9)
    ax[idx].set_xlabel("")
ax[-1].legend(loc=(1.02,-0.04), title="Profiler")
ax[0].set_yticks([0., 1.0, 2.0, 3.0, 4.0, 5])
ax[0].set_xticks([500, 1000, 1500, 2000])
for a in ax:
    a.tick_params(axis="x", rotation=30)
for axis in ax:
    axis.grid(True, linewidth=0.5)  # decrease linewidth to make grid lines thinner

plt.subplots_adjust(wspace=0.15)
fig.supxlabel("Dataset size", y=-0.26)
ax[0].set_ylabel("Computation time [s]")
plt.savefig(f"{SAVE_FOLDER}/computation_time.pdf", **SAVEFIG_KWARGS)